In [1]:
from langchain_chroma import Chroma
from langchain_community.llms import Ollama
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [2]:
PDF_PATH = 'data/fundamentals-of-foodnutrition-and-diet-therapy.pdf'


loader = PyPDFLoader(PDF_PATH)
data = loader.load()

# Split your data up into smaller documents with Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
splits = text_splitter.split_documents(data)

In [3]:
# create a Chroma vector store
vector_store = Chroma.from_documents(documents=splits, embedding=GPT4AllEmbeddings())
vector_store

In [4]:
retriever = vector_store.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'GPT4AllEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x136430250>)

In [5]:
# Prompt
prompt = hub.pull("rlm/rag-prompt")

In [6]:
# Ollama LLM
llm = Ollama(model='llama3')

In [7]:
# Create a RAG Chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [8]:
# invoke the chain with a query
print(rag_chain.invoke('nutrition for cancer'))

Parenteral or enteral nutrition may be necessary in the early stages to meet the needs of the patient. Cancer patients require special attention and diet plans should include favourite foods, prepared in familiar ways and served attractively. The nutritional status of the patient improves after chemotherapy, and energy demands can increase due to a hypermetabolic state, requiring 45-50 kcal/kg body weight and adequate carbohydrates and fats to provide energy while sparing protein for tissue building and immune system function.
